In [1]:
import pandas as pd
import numpy as np
import datetime as datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
timed_df = pd.read_csv("Fetched_nifty500_fm2019_withDATE.csv")
timed_df.fillna(0, inplace=True)


timed_df['Date'] = pd.to_datetime(timed_df['--date--'])
timed_df.set_index('Date', inplace=True)
timed_df = timed_df.drop(columns='--date--')


# Drop columns where every entry is 0.0
timed_df = timed_df.loc[:, (timed_df != 0).any(axis=0)]


# # # Use the column selection to drop columns where less than the threshold number of values are non-zero
threshold = 0.70 * len(timed_df)
timed_df = timed_df.loc[:, (timed_df != 0).sum() >= threshold]


# Iterate through each column
for col in timed_df.columns:
    # Calculate the mean of the last 10 non-zero values using rolling and mean
    rolling_mean = timed_df[col].replace(0, np.nan).rolling(window=10, min_periods=1).mean()
    
    # Fill zero values with the calculated rolling mean
    timed_df[col] = timed_df.apply(lambda row: row[col] if row[col] != 0 else rolling_mean[row.name], axis=1)

In [4]:
nifty_df = pd.read_csv("nifty.csv")
nifty_df['Date'] = pd.to_datetime(nifty_df['Date'])
nifty_df.set_index('Date', inplace=True)

# Step 1: Calculate Nifty percentage returns for each rolling 6-month period
nifty_returns = nifty_df['Close'].pct_change().rolling(window=126).sum().dropna()
nifty_returns = nifty_returns.max()

# Step 2: Check if each stock meets the condition
retained_stocks = []

for stock in timed_df.columns:
    stock_returns = timed_df[stock].pct_change().rolling(window=252).sum().dropna()
    
    # Check if any 6-month period has at least 50% of Nifty return
    if (stock_returns[stock_returns >= 1].count() >= 5):
        retained_stocks.append(stock)

# Step 3: Retain only the columns of retained stocks in stock_df
timed_df = timed_df[retained_stocks]


In [5]:
timed_df

,APLAPOLLO,AUBANK,AARTIDRUGS,ADANIENT,ADANIGREEN,ADANIPORTS,ADANIPOWER,ATGL,ABCAPITAL,AFFLE,...,VAIBHAVGBL,VTL,VARROC,VEDL,IDEA,WELCORP,WELSPUNLIV,WIPRO,ZENSARTECH,ECLERX
Date,,,,,,,,,,,,,,,,,,,,,
2019-09-19,127.620003,322.399994,111.312500,142.550003,50.150002,361.899994,61.900002,135.250000,89.400002,188.029999,...,169.960007,188.119995,431.100006,146.149994,5.45,127.000000,50.400002,241.500000,214.850006,334.799988
2019-09-20,130.934998,322.250000,116.375000,149.100006,51.599998,380.700012,63.849998,133.350006,94.250000,194.589996,...,170.759995,183.429993,492.049988,158.949997,5.85,132.750000,52.049999,245.949997,215.250000,321.733337
2019-09-23,140.604996,341.250000,119.737503,152.399994,52.200001,410.950012,65.900002,135.350006,93.199997,199.960007,...,171.990005,187.410004,487.899994,157.899994,5.60,137.750000,56.950001,239.350006,210.750000,312.433319
2019-09-24,141.744995,334.975006,120.724998,148.050003,52.700001,413.850006,67.599998,133.100006,90.849998,196.880005,...,170.139999,190.830002,470.950012,159.949997,5.60,138.699997,57.299999,240.699997,210.000000,303.399994
2019-09-25,139.139999,322.024994,116.487503,147.550003,52.549999,414.799988,68.449997,131.449997,92.550003,216.279999,...,170.880005,191.619995,453.950012,156.199997,5.30,134.949997,54.599998,238.800003,210.100006,296.066681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-02,1478.650024,629.000000,509.350006,3157.449951,1674.250000,1261.449951,559.400024,999.250000,180.350006,1230.599976,...,481.000000,422.200012,600.349976,272.549988,14.05,590.799988,156.250000,483.149994,560.950012,2687.050049
2024-02-05,1435.599976,622.599976,520.549988,3173.449951,1668.650024,1258.550049,546.299988,989.049988,178.199997,1195.000000,...,471.799988,427.799988,600.049988,273.600006,13.75,595.700012,152.300003,482.350006,564.400024,2624.550049
2024-02-06,1402.849976,616.500000,538.650024,3203.750000,1721.650024,1273.099976,549.700012,994.950012,180.149994,1184.250000,...,489.500000,427.450012,605.599976,283.000000,14.25,596.099976,152.449997,499.649994,592.150024,2667.949951


In [6]:
# Assuming timed_df is your DataFrame indexed by date

# Set the desired start and end dates
start_date = '2019-09-19'
end_date = '2024-08-02'

# Use loc to select rows within the specified date range
timed_df = timed_df.loc[start_date:end_date]

# Set the desired number of columns to keep
num_columns_to_keep = 100 #Replace with your desired number

# first num columns
# timed_df = timed_df.iloc[:, :num_columns_to_keep]

# last num columns 
# timed_df = timed_df.iloc[:, -num_columns_to_keep:]

#random num columns
timed_df = timed_df.sample(n=num_columns_to_keep, axis=1)


timed_df.columns

Index(['PERSISTENT', 'JSL', 'STLTECH', 'COCHINSHIP', 'NBCC', 'JYOTHYLAB',
       'MAXHEALTH', 'KEC', 'TATAPOWER', 'AFFLE', 'CENTRALBK', 'UCOBANK',
       'MAZDOCK', 'NIACL', 'RTNINDIA', 'AARTIDRUGS', 'CHOLAFIN', 'BSOFT',
       'BALRAMCHIN', 'FINCABLES', 'NMDC', 'ENGINERSIN', 'ELGIEQUIP', 'HUDCO',
       'RHIM', 'HINDCOPPER', 'POLYMED', 'BAJAJFINSV', 'TEJASNET', 'TATACHEM',
       'BALAMINES', 'IBULHSGFIN', 'USHAMART', 'FACT', 'PRESTIGE', 'BCG',
       'LINDEINDIA', 'SKFINDIA', 'JSWSTEEL', 'RCF', 'RVNL', 'ALKYLAMINE',
       'PRINCEPIPE', 'KPRMILL', 'SUNDRMFAST', 'NAUKRI', 'TIINDIA', 'BSE',
       'OIL', 'SONATSOFTW', 'WELCORP', 'DIXON', 'ASHOKLEY', 'JUSTDIAL',
       'KNRCON', 'TATASTEEL', 'LAURUSLABS', 'JKCEMENT', 'GODREJPROP', 'LTTS',
       'CGCL', 'NLCINDIA', 'GAEL', 'GLENMARK', 'INFIBEAM', 'PRSMJOHNSN',
       'LAXMIMACH', 'SAIL', 'ZENSARTECH', 'BDL', 'CIEINDIA', 'ADANIPORTS',
       'OLECTRA', 'SWSOLAR', 'CGPOWER', 'CENTURYTEX', 'PNBHOUSING',
       'TATAMOTORS', 'CUMMINSIND', '

In [7]:
timed_df

,PERSISTENT,JSL,STLTECH,COCHINSHIP,NBCC,JYOTHYLAB,MAXHEALTH,KEC,TATAPOWER,AFFLE,...,CYIENT,POLYPLEX,TRENT,PRAJIND,BHARATFORG,PEL,IDFCFIRSTB,SUZLON,LEMONTREE,IRCTC
Date,,,,,,,,,,,,,,,,,,,,,
2019-09-19,594.650024,33.549999,153.149994,171.350006,34.000000,156.800003,NaN,250.449997,64.750000,188.029999,...,444.500000,433.950012,455.450012,113.900002,394.149994,1722.062744,40.599998,2.950000,53.000000,NaN
2019-09-20,599.049988,33.700001,167.949997,166.750000,37.500000,162.050003,NaN,275.299988,67.050003,194.589996,...,450.649994,457.399994,489.799988,116.349998,424.600006,1804.351685,43.099998,3.050000,53.799999,NaN
2019-09-23,619.400024,34.299999,176.399994,174.824997,39.299999,168.399994,NaN,288.450012,67.199997,199.960007,...,471.299988,456.399994,508.000000,121.050003,452.350006,1801.667236,43.299999,3.100000,59.700001,NaN
2019-09-24,600.849976,34.700001,169.149994,175.475006,39.349998,168.750000,NaN,280.350006,66.400002,196.880005,...,465.299988,457.700012,515.400024,119.349998,460.200012,1804.742065,42.799999,3.000000,58.450001,NaN
2019-09-25,570.099976,36.049999,156.100006,172.000000,37.049999,165.600006,NaN,275.850006,63.799999,216.279999,...,447.399994,441.750000,517.849976,119.449997,442.299988,1780.729004,42.950001,2.950000,58.549999,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-02,8502.599609,596.549988,140.350006,905.200012,168.649994,509.700012,766.400024,666.200012,389.799988,1230.599976,...,2109.750000,1007.049988,3126.050049,477.250000,1238.849976,921.849976,82.550003,48.599998,142.050003,967.500000
2024-05-02,8553.599609,606.599976,140.399994,890.650024,166.100006,496.200012,789.549988,652.200012,394.000000,1195.000000,...,2169.449951,985.900024,3027.149902,472.950012,1271.599976,896.900024,83.250000,48.250000,139.850006,946.650024
2024-06-02,8642.000000,616.099976,140.649994,884.500000,157.449997,498.600006,821.400024,659.099976,392.950012,1184.250000,...,2201.699951,984.349976,3035.000000,508.049988,1285.500000,902.400024,83.000000,48.000000,142.949997,965.250000


In [8]:
# Show daily returns
returns = timed_df.pct_change()
returns.fillna(0, inplace=True)
returns.replace([np.inf, -np.inf], 0, inplace=True)

returns

,PERSISTENT,JSL,STLTECH,COCHINSHIP,NBCC,JYOTHYLAB,MAXHEALTH,KEC,TATAPOWER,AFFLE,...,CYIENT,POLYPLEX,TRENT,PRAJIND,BHARATFORG,PEL,IDFCFIRSTB,SUZLON,LEMONTREE,IRCTC
Date,,,,,,,,,,,,,,,,,,,,,
2019-09-19,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2019-09-20,0.007399,0.004471,0.096637,-0.026846,0.102941,0.033482,0.000000,0.099221,0.035521,0.034888,...,0.013836,0.054038,0.075420,0.021510,0.077255,0.047785,0.061576,0.033898,0.015094,0.000000
2019-09-23,0.033971,0.017804,0.050313,0.048426,0.048000,0.039185,0.000000,0.047766,0.002237,0.027597,...,0.045823,-0.002186,0.037158,0.040395,0.065356,-0.001488,0.004640,0.016393,0.109665,0.000000
2019-09-24,-0.029948,0.011662,-0.041100,0.003718,0.001272,0.002078,0.000000,-0.028081,-0.011905,-0.015403,...,-0.012731,0.002848,0.014567,-0.014044,0.017354,0.001707,-0.011547,-0.032258,-0.020938,0.000000
2019-09-25,-0.051178,0.038905,-0.077150,-0.019803,-0.058450,-0.018667,0.000000,-0.016051,-0.039157,0.098537,...,-0.038470,-0.034848,0.004753,0.000838,-0.038896,-0.013306,0.003505,-0.016667,0.001711,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-02,0.024934,0.056589,-0.014050,-0.006966,0.190611,0.002952,-0.045698,0.008172,0.002443,0.013507,...,0.087977,-0.009248,0.007493,-0.039352,0.007564,0.030288,-0.010192,0.008299,0.023415,0.004360
2024-05-02,0.005998,0.016847,0.000356,-0.016074,-0.015120,-0.026486,0.030206,-0.021015,0.010775,-0.028929,...,0.028297,-0.021002,-0.031637,-0.009010,0.026436,-0.027065,0.008480,-0.007202,-0.015487,-0.021550
2024-06-02,0.010335,0.015661,0.001781,-0.006905,-0.052077,0.004837,0.040339,0.010580,-0.002665,-0.008996,...,0.014866,-0.001572,0.002593,0.074215,0.010931,0.006132,-0.003003,-0.005181,0.022167,0.019648


In [9]:
# Annualized covariance matrix

# cov_matrix_annual = returns.cov()*252
cov_matrix_annual = returns.cov()*246
cov_matrix_annual.fillna(0, inplace=True)
cov_matrix_annual

,PERSISTENT,JSL,STLTECH,COCHINSHIP,NBCC,JYOTHYLAB,MAXHEALTH,KEC,TATAPOWER,AFFLE,...,CYIENT,POLYPLEX,TRENT,PRAJIND,BHARATFORG,PEL,IDFCFIRSTB,SUZLON,LEMONTREE,IRCTC
PERSISTENT,0.130767,0.034965,0.035580,0.023331,0.039765,0.019608,0.012344,0.026167,0.033829,0.032138,...,0.039259,0.029949,0.029677,0.037664,0.038087,0.044233,0.038061,0.015449,0.036400,0.023681
JSL,0.034965,0.267540,0.077074,0.041527,0.085049,0.041661,0.015546,0.041062,0.070663,0.054019,...,0.044609,0.065895,0.054875,0.074184,0.056503,0.075158,0.084429,0.049940,0.050249,0.047649
STLTECH,0.035580,0.077074,0.219436,0.044616,0.076946,0.041960,0.017486,0.045555,0.060124,0.048984,...,0.047927,0.057734,0.041225,0.069436,0.050725,0.069017,0.068721,0.052691,0.049368,0.050346
COCHINSHIP,0.023331,0.041527,0.044616,0.184443,0.077232,0.037162,0.014721,0.037064,0.043012,0.038526,...,0.027348,0.038771,0.041699,0.055044,0.041558,0.038831,0.059400,0.050277,0.052497,0.039988
NBCC,0.039765,0.085049,0.076946,0.077232,0.254915,0.051828,0.015364,0.053238,0.092013,0.056728,...,0.048959,0.050432,0.055703,0.087356,0.071450,0.074282,0.102584,0.086224,0.078558,0.069789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PEL,0.044233,0.075158,0.069017,0.038831,0.074282,0.043489,0.010784,0.046489,0.063007,0.053759,...,0.040951,0.053854,0.061408,0.067639,0.065499,0.265439,0.085189,0.057367,0.053063,0.057955
IDFCFIRSTB,0.038061,0.084429,0.068721,0.059400,0.102584,0.045761,0.017365,0.049585,0.078336,0.052168,...,0.047022,0.050038,0.061665,0.075432,0.074024,0.085189,0.173374,0.057116,0.067484,0.056056
SUZLON,0.015449,0.049940,0.052691,0.050277,0.086224,0.030861,0.016893,0.023967,0.051961,0.034336,...,0.017573,0.038230,0.030642,0.063615,0.037869,0.057367,0.057116,0.378772,0.050112,0.048821
LEMONTREE,0.036400,0.050249,0.049368,0.052497,0.078558,0.031923,0.019576,0.040705,0.058268,0.049427,...,0.035094,0.035062,0.050223,0.061227,0.064929,0.053063,0.067484,0.050112,0.225708,0.055983


In [10]:
weights = np.full(timed_df.shape[1], 1 / timed_df.shape[1])
weights

array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01])

In [11]:
# Calculate PORTFOLIO variance
port_variance = np.dot(weights.T, np.dot(cov_matrix_annual, weights))
port_variance

0.047554498471449554

In [12]:
# Calculate PORTFOLIO volatility (std. dev)
port_volatility = np.sqrt(port_variance)
port_volatility

0.21806993940350777

In [13]:
# Annual portfolio return
portfolioSimpleAnnualReturn = np.sum(returns.mean() * weights) * 252
portfolioSimpleAnnualReturn

0.46436190136244354

In [14]:
# DISPLAY

percent_var = str(round(port_variance, 2)*100) + "%"
percent_vol = str(round(port_volatility, 2)*100) + "%"
percent_ret = str(round(portfolioSimpleAnnualReturn, 2)*100) + "%"

print ("Expected Annual Return: " + percent_ret)
print ("Annual Volatility: " + percent_vol)
print ("Annual Variance: " + percent_var)

Expected Annual Return: 46.0%
Annual Volatility: 22.0%
Annual Variance: 5.0%


# OPTIMIZATION

In [15]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [16]:
mu = expected_returns.mean_historical_return(timed_df, frequency=252)

# mu = expected_returns.ema_historical_return(timed_df)

# mu = expected_returns.return_model(timed_df)


# mu = expected_returns.capm_return(timed_df)


mu.fillna(0, inplace = True)
mu.replace(np.inf, 0.0, inplace=True)

In [17]:
# S = risk_models.sample_cov(timed_df)

S = risk_models.exp_cov(timed_df)

# S = risk_models.CovarianceShrinkage(timed_df)

# S = (S + S.T) / 2

S.fillna(0, inplace = True)
S.replace(np.inf, 0.0, inplace=True)

In [18]:
# S_normalized
# S_f = (S + S.T) / 2

#S_reg
S_f = S + 1e-6 * np.eye(S.shape[0])


In [19]:
# ef = EfficientFrontier(mu, S)
ef = EfficientFrontier(mu, S_f, solver="ECOS")
# solvers: ECOS, OSQP


# ef.add_constraint(objective_functions.portfolio_return, return_target=0.25)
# ef.add_constraint(objective_functions.portfolio_volatility, args=[S_f], risk_target=0.25)
# ef.add_objective(custom_objective, args=(mu, S_f))


# ef.add_constraint()

weights = ef.max_sharpe()
# weights = ef.min_volatility()

# Maximize Return for a Target Volatility:
# weights = ef.efficient_return(target_return=0.30)

# Minimize Volatility for a Target Return:
# weights = ef.efficient_risk(target_volatility=0.30)

# weights = ef.add_sector_constraints







# --------------------------------------------------------------------------


# ef_efficient_return = EfficientFrontier(mu, S, solver="ECOS")
# weights_efficient_return = ef_efficient_return.efficient_return(target_return=0.2)

# # Minimum Volatility Portfolio
# ef_min_volatility = EfficientFrontier(mu, S)
# ef_min_volatility.min_volatility()
# weights_min_volatility = ef_min_volatility.clean_weights()

# # Combine the weights
# alpha = 0.4
# combined_weights = np.array([alpha * weights_efficient_return[asset] + (1 - alpha) * weights_min_volatility[asset]
#                              for asset in timed_df.columns])

# # Create a new EfficientFrontier instance with the combined weights
# ef_combined = EfficientFrontier(mu, S, solver="ECOS")
# ef_combined.weights = combined_weights

# # Use clean_weights on the combined weights

refined_weights = ef.clean_weights()


In [20]:
print ()
print ()
print ("-----PORTFOLIO PERFORMANCE-----")
print ()


ef.portfolio_performance(verbose = True)


print ()
print ()
print ("-----ASSET ALLOCATION-----")
print ()


refined_weights = {key: value for key, value in refined_weights.items() if value != 0.0}

# Normalize the percentages
total_weight = sum(refined_weights.values())
refined_weights_percent = {key: (value / total_weight) * 100 for key, value in refined_weights.items()}
assest = []
weight = {}
for key, value in refined_weights_percent.items():
    assest.append(key)
    weight[key] = value
    print(f"{key}: {value:.2f}%")




-----PORTFOLIO PERFORMANCE-----

Expected annual return: 90.7%
Annual volatility: 14.5%
Sharpe Ratio: 6.10


-----ASSET ALLOCATION-----

PERSISTENT: 18.97%
JSL: 9.10%
MAXHEALTH: 7.98%
MAZDOCK: 5.30%
RTNINDIA: 4.12%
ELGIEQUIP: 1.13%
POLYMED: 1.42%
FACT: 2.63%
LINDEINDIA: 2.55%
ALKYLAMINE: 3.32%
SUNDRMFAST: 0.50%
BSE: 0.85%
OIL: 1.39%
SONATSOFTW: 2.06%
DIXON: 4.00%
CGCL: 3.84%
CGPOWER: 16.20%
TATAMOTORS: 3.72%
BLS: 1.23%
POWERINDIA: 1.97%
POLYCAB: 1.19%
TRENT: 2.72%
SUZLON: 3.82%


In [20]:
newTimeDf = timed_df[[i for i in weight.keys()]]

In [21]:
weight

{'APARINDS': 3.698963010369896,
 'BSOFT': 19.566804331956682,
 'ADANIGREEN': 34.355656443435564,
 'JBMA': 42.37857621423785}

In [22]:
import math


def DiscreteAllocation(timed_df, weight, investAmount, startDate):
    reminder = 0
    newWeights = {}
    for key, value in weight.items():
        allocatedPrice = value*investAmount*0.01
        # Use Start date in iloc
        units = math.floor(allocatedPrice / timed_df[key][startDate])
        print(units)
        rem: pd.Series = allocatedPrice - units * timed_df[key][startDate]
        print(rem)
        newWeights[key] = {"price": timed_df[key][startDate], "units": units, "allocated": (
            value*investAmount*0.01), "reminder": rem}
        reminder += rem

    return reminder, newWeights

r, weights = DiscreteAllocation(newTimeDf, weight,100000, start_date)

6
471.2629371698963
293
9.05433195668229
685
2.905395385569136
711
31.41524016785843


In [23]:
weights

{'APARINDS': {'price': 537.9500122,
  'units': 6,
  'allocated': 3698.963010369896,
  'reminder': 471.2629371698963},
 'BSOFT': {'price': 66.75,
  'units': 293,
  'allocated': 19566.804331956682,
  'reminder': 9.05433195668229},
 'ADANIGREEN': {'price': 50.15000153,
  'units': 685,
  'allocated': 34355.65644343557,
  'reminder': 2.905395385569136},
 'JBMA': {'price': 59.56000137,
  'units': 711,
  'allocated': 42378.57621423786,
  'reminder': 31.41524016785843}}

In [24]:
from collections import defaultdict


def BackTest(df, startDate, duration, weights):
    """
    duration: in days
    startDate: starting date string
    weights: weights dict 
    """
    window = 2 #month
    start = datetime.datetime.strptime(startDate, "%Y-%m-%d")
    end = start + datetime.timedelta(days=30*window)

    end_stop_date = start + datetime.timedelta(days=duration)
    x = defaultdict(dict)
    c = 1
    while end < end_stop_date:
        end = start + datetime.timedelta(days=30*window)
        temp = df.loc[start:end, :]

        # print(temp.iloc[-1,0])
        for i in weights.keys():
            x[i][c] = {
                "date_start": str(temp[i].iloc[0:].index[0])[:10],
                "date_end": str(temp[i].iloc[-1:].index[0])[:10],
                "date_start_price": temp[i].iloc[0],
                "date_end_price": temp[i].iloc[-1]
            }
        for key, values in x.items():
            st = values[c]["date_start_price"]
            en = values[c]["date_end_price"]

            st_price = st * weights[key]["units"]
            en_price = en * weights[key]["units"]
            pct_cng = (en_price - st_price)/st_price * 100
            values[c]["st_price"] = st_price
            values[c]["en_price"] = en_price

            values[c]["pct_change"] = pct_cng
        start = end
        c += 1

    return x, c-1

In [25]:
window, total_windows = BackTest(newTimeDf,"2019-09-20", 1000, weights)
print(window, total_windows)

KeyError: Timestamp('2020-01-18 00:00:00')

In [26]:
def PercentChange(window, totalWindows):
    pctChange = []
    endDate = []
    for part in range(1, totalWindows+1):
        startPrice = endPrice = 0
        end = None
        for key, value in window.items():
            cycle = window[key].get(part)
            startPrice += cycle['st_price']
            endPrice += cycle['en_price']
            # print(part , cycle['date_end'])
            end = cycle['date_end']
        endDate.append(end)
        pctChange.append(((endPrice - startPrice)/startPrice * 100))
    return pctChange, endDate

In [27]:
portfolioPercentChange, endDates = PercentChange(window, total_windows)

NameError: name 'window' is not defined

In [ ]:
portfolio = pd.DataFrame({
    'Date': endDates,
    'PctChange': portfolioPercentChange
})


In [ ]:
print(portfolio)

In [ ]:
# plt.plot(data=portfolio)
ax = portfolio.plot(x="Date", y="PctChange", kind="scatter", figsize=[12,6], style='b', rot=90)
portfolio.plot(x="Date", y="PctChange", kind="line", ax=ax, style='b', rot=90)

In [ ]:
nifty = pd.read_csv("nifty.csv")

In [ ]:
nifty['Date'] = pd.to_datetime(nifty['Date'])
nifty.set_index('Date', inplace=True)

# Drop columns where every entry is 0.0
nifty = nifty.loc[:, (nifty != 0).any(axis=0)]

# # # Use the column selection to drop columns where less than the threshold number of values are non-zero
threshold = 0.70 * len(nifty)
nifty = nifty.loc[:, (nifty != 0).sum() >= threshold]
nifty = nifty.iloc[::-1]

In [ ]:
# rename Close to Nifty
nifty.rename(columns={'Close': 'nifty'}, inplace=True)

In [ ]:
# reverse the index
nifty = nifty.iloc[::-1]

In [ ]:
# from collections import defaultdict
# def BackTest_Nifty(nData,startDate, duration):
#     window = 2  # month
#     print(type(startDate))
#     start = datetime.datetime.strptime(startDate, "%Y-%m-%d")
#     end = start + datetime.timedelta(days=30*window)

#     end_stop_date = start + datetime.timedelta(days=duration)
#     x = defaultdict(dict)
#     c = 1
#     print(type(end))
#     print(type(end_stop_date))

#     while end < end_stop_date:
#         end = start + datetime.timedelta(days=30*window)
#         temp = nData.loc[start:end, :]
#         # print(temp["Close"])
#         print(temp)
#         x["nifty"][c] = {
#             "date_start": str(temp["Close"].iloc[0:].index[0]),
#             "date_end": str(temp["Close"].iloc[-1:].index[0]),
#             "st_price": temp["Close"].iloc[0],
#             "en_price": temp["Close"].iloc[-1]
#         }
#         start = end
#         c += 1
#     return x, c-1

In [ ]:
r, weights = DiscreteAllocation(nifty, {"nifty": 100.0}, 100000, "2010-01-05")

In [ ]:
weights

In [ ]:
win, total_ = BackTest(nifty, "2010-01-05", 3000, weights)

In [ ]:
win, total_

In [ ]:
niftyPercentChange, niftyendDates = PercentChange(win, total_)

In [ ]:
nifty_portfolio = pd.DataFrame({
    'Date': niftyendDates,
    'niftyPctChange': niftyPercentChange
})

In [ ]:
print(nifty_portfolio)

In [ ]:
# plt.plot(data=portfolio)
nax = nifty_portfolio.plot(x="Date", y="niftyPctChange", kind="scatter",
                    figsize=[12, 6], style='b', rot=90)
nifty_portfolio.plot(x="Date", y="niftyPctChange", kind="line",
                     ax=nax, style='b', rot=90)

In [ ]:
dats = pd.concat([portfolio, nifty_portfolio["niftyPctChange"]], axis=1)

In [ ]:
dats

In [ ]:
x = dats.plot(x="Date", y="PctChange", kind="line", rot=90)
dats.plot(x="Date", y="niftyPctChange",kind="line", ax=x, rot=90)